In [3]:
import time
import requests
import soopProcess
import soopPull

# this will change for sure
ROOT_URL = "http://127.0.0.1:8000"

# put this in env variable pls
AUTH_STR = "Token 4847d6ca5fb338bd0f35531c4609a0d1bf67b5a6"



msu = soopProcess.processor(
    'http://calendar.msu.montana.edu/events/',
    startNum=26500,
    endNum=26505,
    urlType='numbered',
)

bozeMag = soopProcess.processor(
    baseUrl='http://bozemanmagazine.com/events/calendar/' + time.strftime('%Y/%m/%d'),
    linkRegex='/' + time.strftime('%Y/%m/%d') + '.*',
    urlType='crawled',
    linkBase='http://bozemanmagazine.com/events',
    today=True,
)


    #important_weight=100,
    #num_sentences=2


In [4]:
links = bozeMag.check_links()

In [5]:
links[0]

'http://bozemanmagazine.com/events/2018/06/15/63043_gary_small_the_coyote_brothers-28227-1'

In [6]:
msuData = msu.process(
    whenTag='dd',
    locationTag='dd',
    #locationRegex='^(?!.*(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday))',
    descTag='main',
    descClass='col-sm-8',
    important_weight=100,
    num_sentences=2
)

bozeMagData = bozeMag.process(
    whenTag='p',
    whenClass='date',
    descTag='div',
    descClass='description',
    # locationTag='address',
    # locationClass='location_text',
    important_weight=100,
    num_sentences=2
)


In [7]:
bozeMagData

,lxml,outUrl,text,food,title,day,when,location,details
1,<!DOCTYPE html> <!--[if lt IE 7]> <html class=...,http://bozemanmagazine.com/events/2018/06/15/6...,"Read, sing, talk, play, write together! Presch...",[free],Preschool Storytime - Belgrade Community Library,6/15/2018,"Every Friday until Jul. 27, 11:15am-noon;...",None,"Preschool storytime includes stories, songs, a..."
9,<!DOCTYPE html> <!--[if lt IE 7]> <html class=...,http://bozemanmagazine.com/events/2018/06/15/6...,Wild Sheep Foundation is partnering with South...,"[bbq, free]","Gear, Beer, and BBQ at Wild Sheep Foundation -...",6/15/2018,Fri. Jun. 15 3-7pm...,None,Wild Sheep Foundation is partnering with South...
11,<!DOCTYPE html> <!--[if lt IE 7]> <html class=...,http://bozemanmagazine.com/events/2018/06/15/6...,An evening of music performed by local musicia...,[coffee],Open Mic Night - Wild Joe's Coffee Spot,6/15/2018,Sat. Jun. 2 6pm...,None,"Bring your guitar, sitar, zither, poetry, come..."


In [32]:
postData = bozeMagData.loc[1]

In [33]:
postData

lxml        <!DOCTYPE html>
<!--[if lt IE 7]> <html class=...
outUrl      http://bozemanmagazine.com/events/2018/06/15/6...
text        Read, sing, talk, play, write together! Presch...
food                                                   [free]
title        Preschool Storytime - Belgrade Community Library
day                                                 6/15/2018
when             Every Friday until Jul. 27, 11:15am-noon;...
location                                                 None
details     Preschool storytime includes stories, songs, a...
Name: 1, dtype: object

In [11]:
ROOT_URL = "http://stewfinder-backend.us-west-2.elasticbeanstalk.com"
AUTH_STR = "Token 1eefcbd33b05b8c56182ae2f953bdd7a33f367b4"

In [34]:
import json
import requests
when = postData['when']
day = postData['day']
details = postData['details']
location = postData['location']
food = postData['food']
outUrl = postData['outUrl']
title = postData['title']
obj = {
    'title': title,
    'details': details,
    'food': food,
    'day': day,
    'outUrl': outUrl,
    'when': when,
    'location': location
}
data = json.dumps(obj)

In [38]:
with requests.session() as client:
    for row in postData:
        client.post(
            ROOT_URL + '/api/soops/',
            data=data,
            headers={'Authorization': AUTH_STR}
        )

str